In [134]:
import sys; sys.path.append('./stepik-dl-nlp')

import re

import warnings
warnings.filterwarnings('ignore')

from sklearn.datasets import fetch_20newsgroups
from sklearn.metrics import accuracy_score

from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.preprocessing import StandardScaler
import numpy as np


import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import collections

import torch
from torch import nn
from torch.nn import functional as F

import dlnlputils
from dlnlputils.data import tokenize_text_simple_regex, tokenize_corpus, build_vocabulary, \
    vectorize_texts, SparseFeaturesDataset
from dlnlputils.pipeline import train_eval_loop, predict_with_model, init_random_seed

init_random_seed()


# Закон Ципфа  
$$f(rank, Z(s, N)) = \frac{1}{Z(s, N)*rank^{s}}$$  

## Z-константа  
$$Z(s, N) = \sum\limits_{i=1}^{N}i^{-s}$$  

In [85]:
# словарь - {слово: ранг встречаемости}
vocabl = {'a':1, 'b':2, 'c':3}
# затухание
s = 2

In [86]:
def calc_zipf(entry_dict, s):
    n = len(entry_dict)
    # z - нормировочная константа
    z = sum([i**(-s) for i in range(1, n+1)])
    zipf_l = []
    for i in entry_dict.keys():    
        zipf_l.append((z**(-1))*(1/entry_dict.get(i)**s))
    return(zipf_l)

In [87]:
calc_zipf(vocabl, s)

[0.7346938775510203, 0.18367346938775508, 0.08163265306122448]

## Написать регулярку
[stepik question link](https://stepik.org/lesson/256724/step/3?unit=237035)

В этом задании мы предлагаем вам собрать регулярное выражение из "деталей" так, чтобы оно выделяло в отдельные токены знаки препинания, числа и слова.

А именно:

Числа с плавающей точкой вида 123.23 выделяются в один токен. Десятичным разделителем может быть точка или запятая.
Число может быть отрицательным: иметь знак -123.4−123.4
Целой части числа может вовсе не быть: последовательности  -0.15−0.15  и  -.15−.15   означают одно и то же число.
При этом числа с нулевой дробной частью не допускаются:  строка "12345.12345." будет разделена на два токена "12345" и "."
Идущие подряд знаки препинания выделяются каждый в отдельный токен.
Наконец множество букв в словах ограничивается только кириллическим алфавитом (33 буквы, включая букву ё).
Обратите внимание, что в результате токенизации не должно получаться пустых токенов.

Вы можете использовать следующие тесты для отладки своего регулярного выражения:

|  Текст  |  Результат  |  
|---------|-------------|
|Контактный телефон: 123123.  |контактный телефон : 123123 .|
|Что-нибудь надо придумать.  |что - нибудь надо придумать .|
|Значение числа Е=2.7182.  |значение числа е = 2.7182 .|
|Демон123, как тебя зовут в реале?  |демон 123 , как тебя зовут в реале ?|
|-1-.15=-1.15  |-1 -.15 = -1.15|
|- 1 - .15 = -1.15  |- 1 - .15 = -1.15|
|Какого ;%:?* тут происходит?  |какого ; % : ? * тут происходит ?|
 

Детали "конструктора", из которых можно собрать решение задачи:  

- [а-яё]+          // ненулевая последовательность любых букв русского алфавита.
- -?\d*[.,]?\d+   // возможно знак, возможная целая часть числа, возможно, десятичный знак и остальная часть числа.
- \S                // любой символ, кроме разделителей (пробелов, переносов строк)
- |     //  | отвечает за выбор из двух паттернов, например: [а-яё]+|\d последовательность букв или одна цифра.

In [88]:
# answer

import re
import sys

# модифицированное выражение
TOKENIZE_RE = re.compile(r'[а-яё]+|-?\d*[.,]?\d+|\S', re.I)

def tokenize(txt):
    return TOKENIZE_RE.findall(txt)

for line in sys.stdin:
    print(' '.join(tokenize(line.strip().lower())))

In [89]:
# check
tokenize('Мама мыла раму.')

['Мама', 'мыла', 'раму', '.']

## Подсчитать частотность токенов
[stepik question link](https://stepik.org/lesson/256724/step/5?unit=237035)

Дана следующая коллекция текстов. Постройте словарь (отображение из строкового представления токенов в их номера) и вектор весов (DF).

$$DF(w) = \frac{DocCount(w, c)}{Size(c)}$$

DF(w) -- частота слова w в коллекции c (отношение количества документов, в которых слово используется, к общему количеству документов).

> Казнить нельзя, помиловать. Нельзя наказывать.

> Казнить, нельзя помиловать. Нельзя освободить.

> Нельзя не помиловать.

> Обязательно освободить.

При токенизации используйте регулярное выражение из семинара: [\w\d]+. После токенизации все токены нужно привести к нижнему регистру. Фильтрацию по частоте не использовать.

Ответ запишите в две строки:

в первой строке - содержимое словаря - список уникальных токенов через пробел в порядке возрастания частоты встречаемости. При одинаковой частоте сортировать по алфавиту.
во второй строке - список весов (DF) токенов, округлённых до 2 знака после запятой и разделённых пробелами, в том же порядке, что и токены в первой строке.

In [90]:
corpus = [
    'Казнить нельзя, помиловать. Нельзя наказывать.',
    'Казнить, нельзя помиловать. Нельзя освободить.',
    'Нельзя не помиловать.',
    'Обязательно освободить.']

In [126]:
TOKENIZE_RE = re.compile(r'[\w\d]+', re.I)

corpus = [x.lower() for x in corpus]

def tokenize(txt):
    return TOKENIZE_RE.findall(txt)

corpus_tokens = [tokenize(x) for x in corpus] 

In [127]:
# corpus_tokens = tokenize_corpus(corpus)

In [128]:
corpus_tokens

[['казнить', 'нельзя', 'помиловать', 'нельзя', 'наказывать'],
 ['казнить', 'нельзя', 'помиловать', 'нельзя', 'освободить'],
 ['нельзя', 'не', 'помиловать'],
 ['обязательно', 'освободить']]

In [129]:
MAX_DF = 1
MIN_COUNT = 1
vocabulary, word_doc_freq = build_vocabulary(corpus_tokens, max_doc_freq=MAX_DF, min_count=MIN_COUNT)

In [130]:
UNIQUE_WORDS_N = len(vocabulary)
print('Количество уникальных токенов', UNIQUE_WORDS_N)
print(list(vocabulary.items())[:10])

Количество уникальных токенов 7
[('нельзя', 0), ('помиловать', 1), ('казнить', 2), ('освободить', 3), ('наказывать', 4), ('не', 5), ('обязательно', 6)]


In [131]:
word_df = [(word, word_doc_freq[i]) for i, (word, _) in enumerate(vocabulary.items())]
word_df

[('нельзя', 0.75),
 ('помиловать', 0.75),
 ('казнить', 0.5),
 ('освободить', 0.5),
 ('наказывать', 0.25),
 ('не', 0.25),
 ('обязательно', 0.25)]

In [133]:
answer = sorted(word_df, key = lambda x: x[1])
answer_1 = []; 
answer_2 = [];

for k, v in list(answer):
    answer_1.append(k)
    answer_2.append(str(v))
    
print(" ".join(answer_1))
print(" ".join(answer_2))

наказывать не обязательно казнить освободить нельзя помиловать
0.25 0.25 0.25 0.5 0.5 0.75 0.75


## Постройте матрицу признаков для текстов
[stepik question link]('https://stepik.org/lesson/256724/step/7?unit=237035')

Постройте матрицу признаков для текстов с шага 5 с использованием словаря и вектора весов, полученного на шаге 5. Используйте взвешивание  
$$lTFIDF = \ln(TF + 1) \cdot IDF$$  

<!-- lTFIDF=ln(TF+1)⋅IDF. -->

Значения признаков следует отмасштабировать так, чтобы для каждого признака его среднее значение по выборке равнялось 0, а среднеквадратичное отклонение 

$$1: x^{scaled}_{i} = \frac{x_{i} - E(x)} {\sigma(x)}$$

В результате масштабирования **для каждого столбца** матрицы признаков среднее должно равняться 0, а среднеквадратичное отклонение 1.

При расчёте среднеквадратического отклонения необходимо использовать скорректированную оценку $$\sigma=\sqrt{\frac{\sum_{i-1}^n(x_i - E(x))^2}{n - 1}}$$
Чтобы получить такую оценку с помощью numpy, необходимо передать параметр `ddof=1`: 

```
feature_matrix = np.zeros((num_docs, num_feats))
feats_std = feature_matrix.std(0, ddof=1)
```

Ответ отформатируйте так, чтобы на каждой строке были признаки одного документа. Порядок столбцов должен соответствовать порядку слов в словаре (как в ответе на шаге 5, по возрастанию df). Столбцы разделяйте одним пробелом. В качестве разделителя целой и дробной части используйте точку или запятую. Округлять значения не обязательно. Решение, при проверке, автоматически округлится до двух знаков. Метод округления - либо "математический", либо свойственный Python rounding half to even strategy, если интересно, посмотрите IEEE 754.

Пример ответа для первых двух документов (до полного ответа не хватает ещё двух строк):

> 1.5  -0.5 -0.5 0.87 -0.76 0.60 0.16  
> -0.5 -0.5 -0.5 0.87 0.18  0.60 0.16

In [135]:
corpus = [
    'Казнить нельзя, помиловать. Нельзя наказывать.',
    'Казнить, нельзя помиловать. Нельзя освободить.',
    'Нельзя не помиловать.',
    'Обязательно освободить.']

#Получаем счетчики слов
TF = CountVectorizer().fit_transform(corpus)

#Строим IDF. К сожалению, в этом задании нам нужно только vectorizer.idf_
#Для стандартных случаев на этой строке все вычисления и заканчиваются.
#Обычно  TFIDF = vectorizer.fit_transform(corpus)
vectorizer = TfidfVectorizer(smooth_idf=False, use_idf=True)
vectorizer.fit_transform(corpus)

## из IDF  в DF
word_doc_freq = 1/np.exp(vectorizer.idf_ - 1)

#TF нормируем и сглаживаем логарифмом (требование задания)
TFIDF = np.log(TF/TF.sum(axis=1)+1) / word_doc_freq 

#Масштабируем признаки
scaledTFIDF = StandardScaler().fit_transform(TFIDF)

#Домножаем на np.sqrt((4-1)/4) для перевода из DDOF(0) в DDOF(1) для 4 текстов
#(требование задания) 
scaledTFIDF *= np.sqrt(3/4)

#Вывод в порядке возрастания DF
for l in scaledTFIDF[:,np.argsort(word_doc_freq)]:
    print (" ".join([ "%.2f" % d for d in l]))

1.50 -0.50 -0.50 0.87 -0.76 0.60 0.16
-0.50 -0.50 -0.50 0.87 0.18 0.60 0.16
-0.50 1.50 -0.50 -0.87 -0.76 0.29 1.04
-0.50 -0.50 1.50 -0.87 1.34 -1.48 -1.36


## Оцените количество памяти  
[stepik question link]('https://stepik.org/lesson/256724/step/11?unit=237035')

Вы занимаетесь тематической классификацией текстов (например, сортируете объявления пользователей по категориям). У вас есть коллекция из 100000 текстов, в которой содержится 10000 уникальных токенов (это размер словаря). Вы хотите построить матрицу признаков для текстов в вашем датасете, чтобы затем обучить логистическую регрессию.  

Построив матрицу, вы обнаруживаете, что 99.5% значений матрицы - нулевые. 

Если бы вы хранили датасет в плотной матрице (например, np.array), то вам бы потребовалось достаточно много памяти (каждое значение признака занимает 4 байта, тип np.float32):

$$dense = |texts| \times |vocab| \times 4bytes = 4 \cdot 10^9 bytes \approx 3814.7Megabytes$$  

(здесь мы считаем, что в мегабайте 1024 килобайт, в килобайте - 1024 байт.)

Вместо этого вы решаете хранить датасет в разреженной матрице в формате COO (coordinate, scipy.sparse.coo_matrix). В этом формате для каждого ненулевого элемента хранится три числа: значение элемента, номер столбца и номер строки. Для хранения координат используется тип np.uint32, 4 байта на каждое значение.

Оцените количество памяти, которое экономится при использовании разреженной матрицы для хранения датасета.

In [140]:
10000/100*0.5

50.0

Ответ:  

$$sparse=∣texts∣×∣vocab∣×(4bytes+4bytes+4bytes)×0.005=6⋅10^{7} bytes$$  
 
$$ SavedMemory=\frac{\text{(dense−sparse)}}{\text{bytes in megabyte}}≈3757.5Megabytes$$